In [36]:
import tensorflow as tf
import numpy as np
import gym
from pylab import *
import scipy.misc

In [27]:
%matplotlib notebook

In [69]:
tf.reset_default_graph()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [70]:
N_ACTIONS = 9

input = tf.placeholder(tf.float32, [None, 84, 84, 4])

with tf.variable_scope('network1'):
    x = tf.layers.conv2d(
            inputs=input,
            filters=32,
            kernel_size=8,
            strides=4,
            activation=tf.nn.relu)

    x = tf.layers.conv2d(
            inputs=x,
            filters=64,
            kernel_size=4,
            strides=2,
            activation=tf.nn.relu)

    x = tf.layers.conv2d(
            inputs=x,
            filters=64,
            kernel_size=3,
            strides=1,
            activation=tf.nn.relu)

    w, h, f = x.shape[1:]
    x = tf.reshape(x, [-1, int(w * h * f)])

    x = tf.layers.dense(
            inputs=x,
            units=512,
            activation=tf.nn.relu)


    a = tf.layers.dense(
            inputs=x,
            units=N_ACTIONS,
            activation=None)

In [81]:
tf.trainable_variables()

[<tf.Variable 'conv2d/kernel:0' shape=(8, 8, 4, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'conv2d_1/kernel:0' shape=(4, 4, 32, 64) dtype=float32_ref>,
 <tf.Variable 'conv2d_1/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv2d_2/kernel:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 <tf.Variable 'conv2d_2/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(3136, 512) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(512, 9) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(9,) dtype=float32_ref>]

In [3]:
env = gym.make('Enduro-v0')
observation = env.reset()

[2017-07-12 18:54:54,029] Making new env: Enduro-v0


In [57]:
observation.shape
action = env.action_space.sample()
os = []
for i in range(4):
    o1, _, _, _ = env.step(action)
    o2, _, _, _ = env.step(action)
    o = np.maximum(o1, o2)
    o = np.mean(o, axis=2)
    o = scipy.misc.imresize(o, (84, 84))
    os.append(o)
os = np.stack(os, axis=-1)

In [74]:
sess.run(tf.global_variables_initializer())

In [76]:
os = np.resize(os, (1, 84, 84, 4))
print(os.shape)
actions = sess.run(a, feed_dict={input: os})

(1, 84, 84, 4)


In [80]:
def with_prob(p):
    if np.random.random() < p:
        return True
    else:
        return False

epsilon = 0.1
    
if with_prob(epsilon):
    action = env.action_space.sample()
else:
    action = np.argmax(actions)

"""
if done:
    y = reward
else:
    y = gamma * max(network_2_output)
"""

'\nif done:\n    y = reward\nelse:\n    y = gamma * max(network_2_output)\n'